In [18]:
from __future__ import print_function
from __future__ import division

import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

In [12]:
dev_visual_features = np.load('../data/dev_visual_features.npy')
dev_annotations = np.load('../data/dev_annotations.npy')
dev_visual_features = preprocessing.minmax_scale(dev_visual_features, feature_range=(0, 1), axis=0)
# dev_visual_features = preprocessing.scale(dev_visual_features, axis=0)
test_visual_features = np.load('../data/test_visual_features.npy')
test_annotations = np.load('../data/test_annotations.npy')
test_visual_features = preprocessing.minmax_scale(test_visual_features, feature_range=(0, 1), axis=0)

In [13]:
train_annotations = dev_annotations
train_visual_features = dev_visual_features
val_annotations = test_annotations
val_visual_features = test_visual_features

In [19]:
# decision tree
model = DecisionTreeRegressor(max_depth=None, min_samples_split=2, random_state=0)
model.fit(train_visual_features, train_annotations[:,0])
dt_pred_0 = model.predict(val_visual_features)
model.fit(train_visual_features, train_annotations[:,1])
dt_pred_1 = model.predict(val_visual_features)

In [20]:
print(mean_squared_error(val_annotations[:,0], dt_pred_0), pearsonr(val_annotations[:,0], dt_pred_0))
print(mean_squared_error(val_annotations[:,1], dt_pred_1), pearsonr(val_annotations[:,1], dt_pred_1))

0.195363475228 (0.082123974716404646, 5.1389596587625255e-10)
0.141548677129 (0.071358156095997036, 6.7445111062923786e-08)


In [21]:
# random forest
model = RandomForestRegressor(n_estimators=10, criterion='mse',
                              max_depth=None, min_samples_split=2,
                              min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                              max_features='auto', max_leaf_nodes=None)

In [22]:
model.fit(train_visual_features, train_annotations[:,0])
rf_pred_0 = model.predict(val_visual_features)
model.fit(train_visual_features, train_annotations[:,1])
rf_pred_1 = model.predict(val_visual_features)

In [23]:
print(mean_squared_error(val_annotations[:,0], rf_pred_0), pearsonr(val_annotations[:,0], rf_pred_0))
print(mean_squared_error(val_annotations[:,1], rf_pred_1), pearsonr(val_annotations[:,1], rf_pred_1))

0.0966436154181 (0.10056000045381651, 2.6081009864262603e-14)
0.100282298283 (-0.038439952406742438, 0.0036709459962088837)
